In [20]:
import numpy as np
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from config import config
from collections import defaultdict

def read_mongo(collection, query={}):
    """ Read from Mongo and Store into DataFrame """

    # Make a query to the specific DB and Collection
    cursor = collection.find(query)
    
    elements = []
    for doc in cursor:
        del doc['_id']
        elements.append(doc)
        
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(elements)
    
    return df

def gerarLinha(repo, modelo, numeroDeIssuesDaRecomendacao):
    return {'projeto': repo, 'modelo': modelo, 'numeroDeIssuesDaRecomendacao': numeroDeIssuesDaRecomendacao, 'numeroDeIssuesTestas': 0, 'nTestes': 0, 'nRecomendacaoDadas': 0, 'totalAcertos': 0, 'totalErros': 0}
    

In [29]:
mongoClient = pymongo.MongoClient(config['DATABASE']['CONNECTION_STRING'])
db = mongoClient[config['DATABASE']['NAME']]
table = []
for col in db.list_collections():
  if not col['name'].endswith('_results'):
    continue
  # projeto (feito)
  # modelo (feito)
  # numeroDeIssuesDaRecomendacao (feito)
  # numeroDeIssuesTestas (feito)
  # nTestes(qnts arquivos corrigidos de verdade) (feito)
  # nRecomendacaoDadas(nTotalDeArquivos)
  # totalAcertos
  # totalErros
  repo = col['name'].replace('_results', '')
  if repo == 'microsoft/vscode':
    continue
  
  linhas = [
    gerarLinha(repo, 'sbert', 3),
    gerarLinha(repo, 'sbert', 5),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'tfidf', 5),
    gerarLinha(repo, 'word2vec', 3),
    gerarLinha(repo, 'word2vec', 5),
  ]

  pos = {
    'sbert': 0,
    'tfidf': 2,
    'word2vec': 4
  }
  
  results = db[col['name']].find({'compare': 'title', 'filtros.goodFirstIssue': 1})
  for result in results:
    tecnica = result['tecnica']
    posicao = pos[tecnica] + 1 if result['topk'] == 5 else pos[tecnica]
    
    linhas[posicao]['numeroDeIssuesTestas'] += 1
    linhas[posicao]['nTestes'] += len(result['arquivos_resolvidos_de_verdade'])
    linhas[posicao]['nRecomendacaoDadas'] += len(result['arquivos_sugeridos'])
    linhas[posicao]['totalAcertos'] += result['acertos']
    linhas[posicao]['totalErros'] += result['erros']
  
  table.extend(linhas)
  
table = pd.DataFrame(table)
table
  

,projeto,modelo,numeroDeIssuesDaRecomendacao,numeroDeIssuesTestas,nTestes,nRecomendacaoDadas,totalAcertos,totalErros
0,facebook/react-native,sbert,3,0,0,0,0,0
1,facebook/react-native,sbert,5,0,0,0,0,0
2,facebook/react-native,tfidf,3,0,0,0,0,0
3,facebook/react-native,tfidf,5,0,0,0,0,0
4,facebook/react-native,word2vec,3,0,0,0,0,0
5,facebook/react-native,word2vec,5,0,0,0,0,0
6,internetarchive/openlibrary,sbert,3,167,648,2318,185,2133
7,internetarchive/openlibrary,sbert,5,165,641,3751,228,3523
8,internetarchive/openlibrary,tfidf,3,167,648,2130,186,1944
9,internetarchive/openlibrary,tfidf,5,165,641,3464,213,3251


In [30]:
mongoClient = pymongo.MongoClient(config['DATABASE']['CONNECTION_STRING'])
db = mongoClient[config['DATABASE']['NAME']]
table = []
for col in db.list_collections():
  if not col['name'].endswith('_results'):
    continue
  # projeto (feito)
  # modelo (feito)
  # numeroDeIssuesDaRecomendacao (feito)
  # numeroDeIssuesTestas (feito)
  # nTestes(qnts arquivos corrigidos de verdade) (feito)
  # nRecomendacaoDadas(nTotalDeArquivos)
  # totalAcertos
  # totalErros
  repo = col['name'].replace('_results', '')
  if repo == 'microsoft/vscode':
    continue
  
  linhas = [
    gerarLinha(repo, 'sbert', 3),
    gerarLinha(repo, 'sbert', 5),
    gerarLinha(repo, 'tfidf', 3),
    gerarLinha(repo, 'tfidf', 5),
    gerarLinha(repo, 'word2vec', 3),
    gerarLinha(repo, 'word2vec', 5),
  ]

  pos = {
    'sbert': 0,
    'tfidf': 2,
    'word2vec': 4
  }
  
  results = db[col['name']].find({'compare': 'title + body', 'filtros.goodFirstIssue': 1})
  for result in results:
    tecnica = result['tecnica']
    posicao = pos[tecnica] + 1 if result['topk'] == 5 else pos[tecnica]
    
    linhas[posicao]['numeroDeIssuesTestas'] += 1
    linhas[posicao]['nTestes'] += len(result['arquivos_resolvidos_de_verdade'])
    linhas[posicao]['nRecomendacaoDadas'] += len(result['arquivos_sugeridos'])
    linhas[posicao]['totalAcertos'] += result['acertos']
    linhas[posicao]['totalErros'] += result['erros']
  
  table.extend(linhas)
  
table = pd.DataFrame(table)
table
  

,projeto,modelo,numeroDeIssuesDaRecomendacao,numeroDeIssuesTestas,nTestes,nRecomendacaoDadas,totalAcertos,totalErros
0,facebook/react-native,sbert,3,1,31,3,0,3
1,facebook/react-native,sbert,5,1,31,3,0,3
2,facebook/react-native,tfidf,3,1,31,0,0,0
3,facebook/react-native,tfidf,5,1,31,0,0,0
4,facebook/react-native,word2vec,3,1,31,0,0,0
5,facebook/react-native,word2vec,5,1,31,0,0,0
6,internetarchive/openlibrary,sbert,3,0,0,0,0,0
7,internetarchive/openlibrary,sbert,5,0,0,0,0,0
8,internetarchive/openlibrary,tfidf,3,0,0,0,0,0
9,internetarchive/openlibrary,tfidf,5,0,0,0,0,0
